### Langchain ChatAPI
https://www.youtube.com/watch?v=e9P7FLi5Zy8&t=65s&ab_channel=DataIndependent

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# environment variables
import os
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
# Initialize openai API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 0. Simple Input/Output Still works
You just need to pass it as a message instead. More on this in a second.

In [30]:
chat = ChatOpenAI(
#     model_name="gpt-3.5-turbo", # Cheaper but less reliable
    model_name="gpt-4",
    temperature=0,
    max_tokens=2000,
    openai_api_key=OPENAI_API_KEY
)

# Initialize with Langchain
#chat = llm(openai_api_key=OPENAI_API_KEY)

In [24]:
message = [HumanMessage(content="What is the name of the most populous state in the USA?")]

chat(message)

AIMessage(content='The most populous state in the USA is California.', additional_kwargs={})

### 1. Chat Messages
* HumanMessage: A message sent from the perspective of the human
* AIMessage: A message sent from the perspective of the AI the human is interacting with
* SystemMessage: A message setting the objectives the AI should follow
* ChatMessage: A message allowing for arbitrary setting of role. You won’t be using this too much

In [7]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content='I hate programming.', additional_kwargs={})

In [8]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="The moon is out")
]
chat(messages)

AIMessage(content='The moon is not out.', additional_kwargs={})

In [9]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="What is the first thing that I said?")
]
chat(messages)

AIMessage(content='You said, "I love programming."', additional_kwargs={})

### Batch Messages

In [10]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Apple")
    ],
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Dog")
    ],
]
chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text='Apples are awesome autumn treats.', generation_info=None, message=AIMessage(content='Apples are awesome autumn treats.', additional_kwargs={}))], [ChatGeneration(text='Daring doggie dug deep.', generation_info=None, message=AIMessage(content='Daring doggie dug deep.', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 64, 'completion_tokens': 14, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo'})

### Prompt Templates

With one or more `MessagePromptTemplates` you can build a `ChatPromptTemplate`

In [11]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template="Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.",
    input_variables=["food_1", "food_2"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [12]:
# Output of system_message_prompt
system_message_prompt.format(food_1="Bacon", food_2="Shrimp")

SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={})

In [13]:
# Make HumanMessagePromptTemplate
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [14]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['food_2', 'text', 'food_1'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['food_1', 'food_2'], output_parser=None, partial_variables={}, template='Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [15]:
chat_prompt_with_values = chat_prompt.format_prompt(food_1="Bacon", \
                                                   food_2="Shrimp", \
                                                   text="I really like food from Germany.")

chat_prompt_with_values.to_messages()

[SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={}),
 HumanMessage(content='I really like food from Germany.', additional_kwargs={})]

In [16]:
response = chat(chat_prompt_with_values.to_messages()).content
print (response)

Great choice! Here are three creative ways to incorporate bacon and shrimp into German cuisine:

1. Bacon-wrapped shrimp skewers: Marinate raw shrimp in olive oil, garlic, and lemon juice for at least 30 minutes. Cut bacon into thin strips and wrap each shrimp with a slice of bacon. Thread the bacon-wrapped shrimp onto skewers and grill until the bacon is crispy and the shrimp is cooked through. Serve with a side of sauerkraut and potato salad for a delicious and hearty German meal.

2. Shrimp and bacon potato soup: Cook diced bacon in a large pot until crispy. Remove the bacon from the pot and set it aside. In the same pot, sauté chopped onion, celery, and carrots until softened. Add diced potatoes, chicken broth, and thyme to the pot and simmer until the potatoes are tender. Stir in cooked shrimp and the reserved bacon and simmer for a few more minutes until the shrimp is heated through. Serve with a slice of hearty German bread for a comforting meal.

3. Bacon and shrimp spaetzle: C

### With Streaming

In [20]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)

resp = chat(chat_prompt_with_values.to_messages())

Great choice! Here are some creative ways to incorporate bacon and shrimp into German cuisine:

1. Bacon and Shrimp Spätzle: Spätzle is a traditional German egg noodle dish. Add cooked shrimp and crispy bacon to the spätzle and toss with butter and herbs for a delicious and hearty meal.

2. Shrimp and Bacon Sauerkraut: Add cooked shrimp and crispy bacon to sauerkraut for a flavorful and protein-packed side dish. Serve with grilled sausages or pork chops for a classic German meal.

3. Bacon and Shrimp Potato Salad: Add cooked shrimp and crispy bacon to a classic German potato salad for a delicious twist on a traditional dish. The smoky bacon and sweet shrimp will add depth of flavor to the creamy potato salad.

4. Shrimp and Bacon Schnitzel: Top a crispy schnitzel with cooked shrimp and crispy bacon for a decadent and flavorful meal. Serve with a side of spaetzle or potato salad for a complete German feast.

5. Bacon and Shrimp Bratwurst: Mix cooked shrimp and crispy bacon into the brat

In [18]:
# For token counting
from langchain.callbacks import get_openai_callback

In [33]:
with get_openai_callback() as cb:
    result = HumanMessage(content="Translate this sentence from English to French. I love programming.")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [34]:
print(result)

content='Translate this sentence from English to French. I love programming.' additional_kwargs={}


In [ ]:
with get_openai_callback() as cb:
    response = agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")